In [1]:
from pyspark.sql.functions import to_date, col, current_date, round, sum as spark_sum

# generate df with todays aggregates
df_today = (
    spark.table("tbl_silver_wrong_currency")
         .filter((col("event_type") == "TRADE") & (col("partition_date") == current_date()))
         .groupBy("partition_date", "symbol", "transaction_type")
         .agg(round(spark_sum(col("price") * col("volume"))/1e6, 2)
              .alias("vol_in_mln_EUR"))
)

# overwrite todays gold partition
(df_today.write
    .format("delta")
    .mode("overwrite")
    .option("replaceWhere", "partition_date = current_date()")
    .saveAsTable("tbl_gold_wrong_volume_per_day")
)

StatementMeta(, 99349f37-1a38-43dc-b993-078d1ade9d46, 3, Finished, Available, Finished)